In [ ]:
import numpy as np
import pickle
# !pip install bcolz
import bcolz

In [ ]:
from collections import Counter
from functools import reduce
# from tqdm.notebook import tqdm
import string
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
import torch
import urllib
import glob
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import nltk
from PIL import Image
from torchvision import transforms
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet

In [ ]:
from PIL import Image
import pandas as pd
import spacy
from torch.nn.utils.rnn import pad_sequence
spacy_eng = spacy.load("en_core_web_sm")

In [ ]:
def read_img(file):
    img_id=[]
    for img in glob.glob(file+'*.jpg'):
        temp=os.path.basename(img)
        img_id.append(temp)
        
    return img_id

In [ ]:
# extract the data from tar files

In [1]:
# use glove
glove_path = '/scratch/varunc/glove/'
glove_zip = glove_path+'glove.6B.zip'

In [ ]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


# Need not run this again and again


words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=glove_path+'6B.300.dat', mode='w')

with open(glove_path+'glove.6B.300d.txt', 'rb') as f:
    pbar = tqdm(total=file_len(glove_path+'glove.6B.300d.txt'))
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
        pbar.update(1)
    
vectors = bcolz.carray(vectors[1:].reshape((400000, 300)), rootdir=glove_path+'6B.300.dat', mode='w')
vectors.flush()
pickle.dump(words, open(glove_path+'6B.300_words.pkl', 'wb'))
pickle.dump(word2idx, open(glove_path+'6B.300_idx.pkl', 'wb'))

In [ ]:
vectors = bcolz.open(glove_path+'6B.300.dat')[:]
words = pickle.load(open(glove_path+'6B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(glove_path+'6B.300_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [ ]:
data_imgid=read_img(flickr_path+"Flickr-8K/Flicker8k_Dataset/")
